In [1]:
from pyspark.sql import SparkSession
import psycopg2
import pandas as pd
import os
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from pyspark import SparkConf

# initialise sparkContext

spark = SparkSession.builder.config("spark.jars", "postgresql-42.2.5.jar")\
    .config("spark.jars.packages","org.postgresql:postgresql:42.2.5")\
    .config('spark.driver.maxResultSize','20g').config("spark.executor.memory", "20g")\
    .config("spark.driver.memory","20g").config("spark.executor.memory","20g")\
    .config("spark.shuffle.file.buffer",'40').config("spark.scheduler.listenerbus.eventqueue.capacity","10000000")\
    .master("local").appName("tinkof-invest").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.sparkContext.setLogLevel("ERROR")
spark


22/05/07 05:11:41 WARN Utils: Your hostname, nikolay resolves to a loopback address: 127.0.1.1; using 192.168.43.45 instead (on interface wlp2s0)
22/05/07 05:11:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nikolay/anaconda3/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nikolay/.ivy2/cache
The jars for the packages stored in: /home/nikolay/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe0321f7-40ae-4b59-a513-74168ea9e03a;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.5 in central
:: resolution report :: resolve 216ms :: artifacts dl 9ms
	:: modules in use:
	org.postgresql#postgresql;42.2.5 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-fe0321f7-40ae-4b59-a513-74168ea9e03a
	confs:

In [2]:
shares_info = spark.createDataFrame(pd.read_parquet('Shares.parquet'))

In [3]:
shares_info

DataFrame[figi: string, ticker: string, class_code: string, isin: string, lot: bigint, currency: string, klong: decimal(9,8), kshort: decimal(9,8), dlong: decimal(9,8), dshort: decimal(9,8), dlong_min: decimal(9,8), dshort_min: decimal(9,8), short_enabled_flag: decimal(9,8), name: string, exchange: string, ipo_date: timestamp, issue_size: bigint, nominal: decimal(13,9), trading_status: bigint, otc_flag: boolean, buy_available_flag: boolean, sell_available_flag: boolean, div_yield_flag: boolean, share_type: bigint, min_price_increment: decimal(10,8), api_trade_available_flag: boolean, uid: string, real_exchange: bigint]

In [18]:
shares_info.write.format('jdbc').option("url","jdbc:postgresql://localhost:5432/test_db")\
    .option("table",'shares_info').option("user",'nikolay').option("password","Andone1")\
    .option("mode","overwrite")

In [12]:
mode = "overwrite"
url = "jdbc:postgresql://localhost:5432/shares"
properties = {"user": "nikolay","password": "Andone1","driver": "org.postgresql.Driver"}


In [4]:
all_candles_day = pd.read_parquet('Shares_market_data2000_2022.parquet')
# all_candles_day.write.format('jdbc').option("url","jdbc:postgresql://localhost:5432/test_db")\
#     .option("table",'candles_day').option("user",'nikolay').option("password","Andone1")\
#     .option("mode","overwrite")

In [5]:
data_now = pd.read_parquet("Shares_market_data2022_now.parquet")

In [6]:
all_data = pd.concat([all_candles_day,data_now])

In [9]:
all_data = spark.createDataFrame(all_data)

In [13]:
all_data.write.jdbc(url=url, table="candles_day", mode=mode, properties=properties)

In [ ]:
shares_info.write.jdbc(url=url, table="shares_info", mode=mode, properties=properties)

In [10]:
print('123')

123
